Importing Important libraries

In [5]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [6]:
df=pd.read_csv('creditcard[1].csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.0727811733098497,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.26615071205963,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.34016307473609,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226008082898,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877736754848451,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


Exploring on the dataset

In [7]:
df.shape

(15957, 31)

In [9]:
df.dtypes

,0
Time,int64
V1,float64
V2,object
V3,float64
V4,float64
V5,float64
V6,float64
V7,float64
V8,float64
V9,float64


In [10]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,1
V4,1
V5,1
V6,1
V7,1
V8,1
V9,1


In [14]:
fraud_count=df["Class"].value_counts()
fraud_rate=100*fraud_count/df.shape[0]
fraud_data=pd.concat([fraud_count,fraud_rate],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,15879,99.511186
1,1.0,74,0.463746


Data Prepration

In [18]:
df_fraud=df[df['Class']==1]
df_not_fraud=df[df['Class']==0]
df_not_fraud_sampled=df_not_fraud.sample(df_fraud.shape[0],replace=False,random_state=101)

df_balanced=pd.concat([df_not_fraud_sampled,df_fraud],axis=0).sample(frac=1,replace=False,random_state=101).reset_index().drop('index',axis=1)
df_balanced

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,2729,-1.753325,1.20667927220279,2.113332,0.800421,-0.129349,0.038541,0.747522,-1.025204,1.517721,...,0.148641,0.944147,-0.228204,0.422821,-0.318327,-0.476384,-2.409573,-0.921618,1.55,0.0
1,12985,1.014262,0.447008129342211,0.963731,2.476404,0.115785,0.663869,-0.228475,0.135007,0.327380,...,-0.265009,-0.665525,0.091318,-0.390824,0.105126,-0.272583,-0.012734,0.020508,53.17,0.0
2,8415,-0.251471,4.31352332575143,-6.891438,6.796797,0.616297,-2.966327,-2.436653,0.489328,-3.371639,...,0.536892,-0.546126,-0.605240,-0.263743,1.539916,0.523574,0.891025,0.572741,1.00,1.0
3,8090,-1.783229,3.40279371307631,-3.822742,2.625368,-1.976415,-2.731689,-3.430559,1.413204,-0.776941,...,0.454032,-0.577526,0.045967,0.461700,0.044146,0.305704,0.530981,0.243746,1.00,1.0
4,3932,-0.500417,1.29316737315872,1.485316,0.188855,0.220677,-0.932238,0.810909,-0.367948,1.219918,...,-0.495491,-0.867529,-0.027278,0.271921,-0.154894,0.009424,0.163123,-0.066235,8.96,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,6746,-0.842628,1.25306561174564,0.449811,-1.570014,3.765948,3.422430,1.863492,-1.077997,2.703464,...,-0.674141,-0.257167,-0.473644,0.951461,0.051191,0.040368,-1.024029,-1.377820,1.00,0.0
144,4492,0.813421,-0.748425754684381,1.340310,0.640720,-1.525057,-0.637926,-0.455720,-0.174873,2.179720,...,-0.107095,-0.260154,-0.041856,0.967640,-0.021689,0.895382,-0.090244,0.050600,200.00,0.0
145,7535,0.026779,4.13246389713003,-6.560600,6.348557,1.329666,-2.513479,-1.689102,0.303253,-3.139409,...,0.370509,-0.576752,-0.669605,-0.759908,1.605056,0.540675,0.737040,0.496699,1.00,1.0
146,4496,1.179814,-0.735532100853499,0.537315,-1.177456,-1.008482,-0.389147,-0.719142,-0.066995,3.383127,...,-0.052222,0.156989,-0.258387,-0.457207,0.521101,0.121850,-0.001897,0.020690,83.00,0.0


In [19]:
#checking the balance of the dataset
fraud_count=df_balanced['Class'].value_counts()
fraud_rate=100*fraud_count/df_balanced.shape[0]
fraud_data=pd.concat([fraud_count,fraud_rate],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']

fraud_data


,Class,Count,Percentage
0,0.0,74,50.0
1,1.0,74,50.0


In [20]:
#train test split
x_train,x_test,y_train,y_test=train_test_split(df_balanced.drop('Class',axis=1),df_balanced['Class'],test_size=0.2,random_state=101)
print(f'''x_train:{x_train.shape}
x_test:{x_test.shape}
y_train:{y_train.shape}
y_test:{x_test.shape}''')

x_train:(118, 30)
x_test:(30, 30)
y_train:(118,)
y_test:(30, 30)


Fitting a random forest model

In [21]:
#logistic model pipe
RandomForestModel=Pipeline([
    ('scaler',StandardScaler()),
    ('classifier', RandomForestClassifier())
])

RandomForestModel.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])

In [22]:
# prediction
y_pred_logis=RandomForestModel.predict(x_test)
y_pred_logis


array([1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.])

In [23]:
#classification
cr=classification_report(y_test,y_pred_logis)
print(cr)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         9
         1.0       1.00      1.00      1.00        21

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [26]:
#saving model
with open('./model.pk1','wb')as fp:
  pickle.dump(RandomForestModel,fp)